In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
from dateutil.parser import isoparse
from datetime import datetime, timedelta
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
courseModalities = pd.read_csv(str(input()))
courseModalities

In [ ]:
courseModalities = courseModalities.loc[~(courseModalities['Modality'] == '(~)')]
courseModalities['short_name'] = courseModalities['Subject Code'] + "-" + courseModalities['Section']
courseModalities.drop(['Subject Code', 'Section', 'Department', 'School'], axis = 1, inplace = True)
courseModalities

In [ ]:
courseModalities['Modality'] = courseModalities['Modality'].apply(lambda modality: modality if modality in ['(FULLY ONLINE)', '(FULLY ONSITE)'] else '(HYBRID)')
courseModalities

In [ ]:
courseList = pd.read_csv(str(input()))
courseList

In [ ]:
courseList = courseList[['canvas_course_id', 'short_name']]
courseList['canvas_course_id'] = courseList['canvas_course_id'].apply(lambda x: 'id_' + str(int(x) + 165820000000000000))
courseList

In [ ]:
courseModalities = courseList.merge(courseModalities, on=['short_name'])
courseModalities

In [ ]:
courseModalities = courseModalities.rename(columns={'canvas_course_id': 'metadata_context_id'})
courseModalities

In [ ]:
courseModalities.to_csv('CourseModalities_Intercession_2022.csv', header = True, index = False)

In [ ]:
courseAssignmentLogs = pd.read_csv(str(input()))
courseAssignmentLogs

In [ ]:
courseModalities = courseModalities.merge(courseAssignmentLogs.rename(columns={'total_logs': 'A_est'}), how = 'left', on=['metadata_context_id'])
courseModalities

In [ ]:
courseQuizLogs = pd.read_csv(str(input()))
courseQuizLogs

In [ ]:
courseModalities = courseModalities.merge(courseQuizLogs.rename(columns={'total_logs': 'Q_est'}), how = 'left', on=['metadata_context_id'])
courseModalities

In [ ]:
courseDiscussionForumLogs = pd.read_csv(str(input()))
courseDiscussionForumLogs

In [ ]:
courseDiscussionForumLogs = courseDiscussionForumLogs.pivot(columns='is_graded', index='metadata_context_id', values='total_logs').reset_index()
courseDiscussionForumLogs = courseDiscussionForumLogs.fillna(0)
courseDiscussionForumLogs

In [ ]:
courseDiscussionForumLogs.columns = ['metadata_context_id', 'UD_est', 'GD_est']
courseDiscussionForumLogs

In [ ]:
courseModalities = courseModalities.merge(courseDiscussionForumLogs, how='left', on=['metadata_context_id'])
courseModalities

In [ ]:
courseLessonLogs = pd.read_csv(str(input()))
courseLessonLogs

In [ ]:
courseModalities = courseModalities.merge(courseLessonLogs.rename(columns={'total_logs': 'L_est'}), how='left', on=['metadata_context_id'])
courseModalities

In [ ]:
courseCommentLogs = pd.read_csv(str(input()))
courseCommentLogs

In [ ]:
courseModalities = courseModalities.merge(courseCommentLogs.rename(columns={'total': 'C_est'}), how='left', on=['metadata_context_id'])
courseModalities

In [ ]:
def addTotalColumns(df, columns):
    result = 0
    for column in columns:
        result = result + df[column]
    return result

courseModalities = courseModalities.fillna(0)
courseModalities['A_est'] = courseModalities.apply(lambda x: addTotalColumns(x, ['A_est', 'GD_est', 'Q_est']), axis = 1)
courseModalities['C_est'] = courseModalities.apply(lambda x: addTotalColumns(x, ['UD_est', 'C_est']), axis = 1)
courseModalities = courseModalities[['metadata_context_id', 'short_name', 'Modality', 'A_est', 'C_est', 'L_est']]
courseModalities

In [ ]:
def checkIfCompleteInfo(df, columns):
    status = True
    for column in columns:
        status = status and (df[column] > 0)
    return status

courseModalities['is_complete'] = courseModalities.apply(lambda x: checkIfCompleteInfo(x, ['A_est', 'C_est', 'L_est']), axis = 1)
courseModalities = courseModalities.loc[courseModalities['is_complete'] == True]
courseModalities.drop(['is_complete'], axis = 1, inplace = True)
courseModalities

In [ ]:
coursesWithGrades = pd.read_csv(str(input()))
coursesWithGrades

In [ ]:
courseModalities = courseModalities.loc[courseModalities['metadata_context_id'].isin(coursesWithGrades['metadata_context_id'].unique())]
courseModalities

In [ ]:
courseModalities.groupby(['Modality'])['metadata_context_id'].count().to_frame('total_classes').reset_index()

In [ ]:
courseModalities['course_code'] = courseModalities['short_name'].apply(lambda x: x.split("-")[0])
courseModalities.drop(['short_name'], axis = 1, inplace = True)
courseModalities

In [ ]:
courseModalities = courseModalities.loc[~(courseModalities['Modality'] == '(FULLY ONSITE)')]
courseModalities

In [ ]:
courseModalitiesCount = courseModalities.groupby(['course_code', 'Modality'])['metadata_context_id'].count().to_frame('total_classes').reset_index()
courseModalitiesCount

In [ ]:
courseModalitiesCount = courseModalitiesCount.pivot(columns = 'Modality', index='course_code', values='total_classes').reset_index()
courseModalitiesCount

In [ ]:
courseModalitiesCount = courseModalitiesCount.dropna()
courseModalitiesCount

In [ ]:
courseModalities = courseModalities.loc[courseModalities['course_code'].isin(courseModalitiesCount['course_code'].unique())]
courseModalities

In [ ]:
courseModalities.drop(['A_est', 'C_est', 'L_est'], axis = 1).to_csv('CourseModalities_Online_Hybrid_Intercession_2022.csv', header = True, index = False)